In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import sklearn.utils.validation

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score
# [핵심 변경 1] 일반 SMOTE 대신 경계선 데이터를 만드는 BorderlineSMOTE 사용
from imblearn.over_sampling import BorderlineSMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# 1. 데이터 로드
print("딥러닝 데이터 준비 중...")
df = pd.read_csv('data/spotify_churn_dataset.csv')

if 'user_id' in df.columns:
    df = df.drop(columns=['user_id'])

# 2. Feature Engineering (ML과 동일하게 적용)
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']

# 3. 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. 전처리
numerical_cols = [
    'age', 'listening_time', 'songs_played_per_day', 'skip_rate', 
    'ads_listened_per_week', 'offline_listening',
    'ad_burden', 'satisfaction_score', 'time_per_song'
]
categorical_cols = ['gender', 'country', 'subscription_type', 'device_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# [핵심 변경 2] BorderlineSMOTE 적용
# 헷갈리는 데이터 위주로 학습 데이터를 생성하여 딥러닝 모델의 판단력을 높임
print("BorderlineSMOTE 적용 중... (시간이 조금 걸릴 수 있습니다)")
bsmote = BorderlineSMOTE(random_state=42, kind='borderline-1')
X_train_res, y_train_res = bsmote.fit_resample(X_train_processed, y_train)

# 5. 모델 설계 (Deep & Wide 구조)
input_dim = X_train_res.shape[1]
model = Sequential([
    Dense(512, input_dim=input_dim, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(256, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(128, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(64, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(32, activation='swish'),
    BatchNormalization(), Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# 6. 학습 시작
print("딥러닝 학습 시작...")
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=0)
checkpoint = ModelCheckpoint('models/spotify_dl_model.h5', monitor='val_accuracy', save_best_only=True, verbose=0)

history = model.fit(
    X_train_res, y_train_res, epochs=150, batch_size=64, validation_split=0.2,
    callbacks=[early_stop, reduce_lr, checkpoint], verbose=1
)

# 7. [핵심 변경 3] 최적의 임계값(Threshold) 찾기
# 딥러닝은 확률값을 뱉어내므로, 0.5가 아닌 최적의 기준선을 찾아 F1 점수를 극대화함
print("최적의 임계값 탐색 중...")
best_model = tf.keras.models.load_model('models/spotify_dl_model.h5')
y_pred_proba = best_model.predict(X_test_processed).flatten()

thresholds = np.arange(0.3, 0.7, 0.01)
best_thresh = 0.5
best_f1 = 0
best_acc = 0

for thresh in thresholds:
    y_pred_temp = (y_pred_proba >= thresh).astype(int)
    f1_temp = f1_score(y_test, y_pred_temp)
    
    if f1_temp > best_f1:
        best_f1 = f1_temp
        best_acc = accuracy_score(y_test, y_pred_temp)
        best_thresh = thresh

# 최종 결과 출력
print("-" * 40)
print(f"🚀 딥러닝 최적 임계값 발견: {best_thresh:.2f}")
print(f"최종 Accuracy: {best_acc:.4f}")
print(f"최종 F1-Score: {best_f1:.4f}")
print("-" * 40)

# 8. 저장
if not os.path.exists('models'): os.makedirs('models')
joblib.dump(preprocessor, 'models/dl_preprocessor.pkl')

metrics_file = 'data/model_metrics.json'
final_metrics = {}

if os.path.exists(metrics_file):
    try:
        with open(metrics_file, 'r') as f:
            final_metrics = json.load(f)
    except:
        pass

# 메트릭 업데이트
final_metrics['Deep Learning (DNN)'] = {
    'Accuracy': best_acc, 
    'F1-Score': best_f1,
    'Best Threshold': float(best_thresh)
}

with open(metrics_file, 'w') as f:
    json.dump(final_metrics, f, indent=4)

print("딥러닝 모델 및 점수 저장 완료.")

딥러닝 데이터 준비 중...
BorderlineSMOTE 적용 중... (시간이 조금 걸릴 수 있습니다)
딥러닝 학습 시작...
Epoch 1/150
134/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5997 - loss: 0.7247

137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6486 - loss: 0.6554 - val_accuracy: 0.2171 - val_loss: 0.7535 - learning_rate: 5.0000e-04
Epoch 2/150
136/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7251 - loss: 0.5526

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7369 - loss: 0.5354 - val_accuracy: 0.4430 - val_loss: 0.7392 - learning_rate: 5.0000e-04
Epoch 3/150
123/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7589 - loss: 0.5097

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7560 - loss: 0.5111 - val_accuracy: 0.6271 - val_loss: 0.6709 - learning_rate: 5.0000e-04
Epoch 4/150
128/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7581 - loss: 0.4946

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7625 - loss: 0.4862 - val_accuracy: 0.7279 - val_loss: 0.6173 - learning_rate: 5.0000e-04
Epoch 5/150
131/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7666 - loss: 0.4762

...

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7777 - loss: 0.4386 - val_accuracy: 0.8850 - val_loss: 0.5815 - learning_rate: 5.0000e-04
Epoch 14/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7840 - loss: 0.4402 - val_accuracy: 0.8791 - val_loss: 0.5883 - learning_rate: 2.5000e-04
Epoch 15/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7823 - loss: 0.4341 - val_accuracy: 0.8804 - val_loss: 0.5880 - learning_rate: 2.5000e-04
Epoch 16/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7836 - loss: 0.4370 - val_accuracy: 0.8814 - val_loss: 0.5907 - learning_rate: 2.5000e-04
Epoch 17/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7792 - loss: 0.4368 - val_accuracy: 0.8814 - val_loss: 0.5955 - learning_rate: 2.5000e-04
Epoch 18/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7878 - loss: 0.4336 - val_accuracy: 0.8846 - val_loss: 0.5792 - learning_rate: 2.5000e-04
Epoch 19/150
129/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy:

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7832 - loss: 0.4333 - val_accuracy: 0.8855 - val_loss: 0.5867 - learning_rate: 1.2500e-04
Epoch 20/150
130/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7892 - loss: 0.4277

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7854 - loss: 0.4329 - val_accuracy: 0.8910 - val_loss: 0.5826 - learning_rate: 1.2500e-04
Epoch 21/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7857 - loss: 0.4321 - val_accuracy: 0.8878 - val_loss: 0.5883 - learning_rate: 1.2500e-04
Epoch 22/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7890 - loss: 0.4291 - val_accuracy: 0.8855 - val_loss: 0.5870 - learning_rate: 1.2500e-04
Epoch 23/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7854 - loss: 0.4317 - val_accuracy: 0.8891 - val_loss: 0.5834 - learning_rate: 1.2500e-04
Epoch 24/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7847 - loss: 0.4304 - val_accuracy: 0.8869 - val_loss: 0.5846 - learning_rate: 6.2500e-05
Epoch 25/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7873 - loss: 0.4267 - val_accuracy: 0.8873 - val_loss: 0.5820 - learning_rate: 6.2500e-05
Epoch 26/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy:

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7855 - loss: 0.4267 - val_accuracy: 0.8914 - val_loss: 0.5762 - learning_rate: 6.2500e-05
최적의 임계값 탐색 중...


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
----------------------------------------
🚀 딥러닝 최적 임계값 발견: 0.55
최종 Accuracy: 0.8135
최종 F1-Score: 0.7447
----------------------------------------
딥러닝 모델 및 점수 저장 완료.
